In [1]:
import math
from os import path, walk
from json import dumps, loads
from pyvis.network import Network
from dataclasses import dataclass, field

from attack import Attack
from defend import DTechnique, DSystem, Defend
from types import SimpleNamespace

In [2]:
d = Defend(Attack(local=True))

graph_config = {
    'tactic': {
        'mass': 100,
        'size': 100,
        'color': '#C00'
    },
    'technique': {
        'mass': 50,
        'size': 50,
        'color': '#C77'
    },
    'subtechnique': {
        'mass': 20,
        'size': 20,
        'color': '#A66'
    },
}
gconf = SimpleNamespace(**graph_config)

In [3]:
def add_node(net, name, config):
    net.add_node(name, **config)

In [21]:
Out = './Poligon'

net = Network(notebook=True, height='1000px', width='1200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')

In [22]:
# Tactics
prev = None
for tac in d.attack.cfg.tactics_order:
    net.add_node(tac, label=tac.replace('-', ' ').title(), **gconf.tactic)
    if prev:
        net.add_edge(prev, tac)
    prev = tac

In [23]:
for tech in d.attack.techniques:
    tac = d.attack.techniques[tech].tactics[0]
    
    parent = d.attack.techniques[tech].get_parent()
    
    if parent and parent not in net.node_ids:
        parent_tac = d.attack.techniques[parent].tactics[0]
        net.add_node(parent, label= d.attack.get_name_by_id(parent), **gconf.technique)
        #net.add_node(parent, color=gconf.technique['color'], mass=gconf.technique['mass'])
        net.add_edge(parent, parent_tac)
    
    if parent:
        net.add_node(tech, label = d.attack.get_name_by_id(tech),  **gconf.subtechnique)
        #net.add_node(tech, color=gconf.subtechnique['color'], mass=gconf.subtechnique['mass'])
        net.add_edge(tech, parent)
    else:
        net.add_node(tech, label = d.attack.get_name_by_id(tech), **gconf.technique)
        #net.add_node(tech, color=gconf.technique['color'], mass=gconf.technique['mass'])
        net.add_edge(tech, tac)

In [24]:
d.attack.techniques['T1542'].tactics

['defense-evasion', 'persistence']

In [27]:
net.show_buttons()
net.show('Test.html')

Test.html


In [43]:
d.attack.techniques['T1055.011'].__dict__.keys()

dict_keys(['name', 'description', 'detection', 'data_sources', 'platforms', 'permissions_required', 'defenses_bypassed', 'id', 'tactics', 'references', 'ref'])